Video Deduplication, also known as Video Copy Detection or Video Identification by Fingerprinting, means that given a query video, you need to find or retrieve the videos with the same content as query video.
The ISC model has been proven to perform well in Video Deduplication [link](https://github.com/ant-research/VCSL). In this tutorial, we show how to fine-tune the ISC operator for better image embedding, which does great help to Video Deduplication in your custom data domain.


### Requirements

If you want to fine tune this operator, make sure your timm version is 0.4.12, and the higher version may cause model collapse during training.

In [ ]:
! python -m pip install tqdm augly timm==0.4.12 pytorch-metric-learning==0.9.99


### Download dataset
ISC is trained using [contrastive learning](https://lilianweng.github.io/posts/2021-05-31-contrastive/), which is a type of self-supervised training. The training images do not require any labels. We only need to prepare a folder `./training_images`, under which a large number of diverse training images can be stored.

In the original training of [ISC21-Descriptor-Track-1st](https://github.com/lyakaap/ISC21-Descriptor-Track-1st), the training dataset is a huge dataset which takes more than 165G space. And it uses [multi-steps training strategy](https://arxiv.org/abs/2104.00298).

In our fine-tune example, to simplification, we prepare a small dataset to run, and you can replace it with your own custom dataset.

In [ ]:
! curl -L https://github.com/towhee-io/examples/releases/download/data/isc_training_image_examples.zip -o ./training_images.zip
! unzip -q -o ./training_images.zip


### Get started to fine-tune
Just call method op.train() and pass in your args.

In [ ]:
import towhee

op = towhee.ops.image_embedding.isc().get_op()
op.train(training_args={
        'train_data_dir': './training_images',
        'distributed': False,
        'output_dir': './output',
        'gpu': 0,
        'epochs': 2,
        'batch_size': 8,
        'init_lr': 0.1
    })

These are each arg infos in training_args:
- output_dir
  - default: './output'
  - metadata_dict: {'help': 'output checkpoint saving dir.'}

- distributed
  - default: False
  - metadata_dict: {'help': 'If true, use all gpu in your machine, else use only one gpu.'}

- gpu
  - default: 0
  - metadata_dict: {'help': 'When distributed is False, use this gpu No. in your machine.'}

- start_epoch
  - default: 0
  - metadata_dict: {'help': 'Start epoch number.'}

- epochs
  - default: 6
  - metadata_dict: {'help': 'End epoch number.'}

- batch_size
  - default: 128
  - metadata_dict: {'help': 'Total batch size in all gpu.'}

- init_lr
  - default: 0.1
  - metadata_dict: {'help': 'init learning rate in SGD.'}

- train_data_dir
  - default: None
  - metadata_dict: {'help': 'The dir containing all training data image files.'}



### Load trained model

In [ ]:
new_op = towhee.ops.image_embedding.isc(checkpoint_path='./output/checkpoint_epoch0001.pth.tar').get_op()


### Your custom training
Your can change [training script](https://towhee.io/image-embedding/isc/src/branch/main/train_isc.py) in your custom way.
Or your can refer to the [original repo](https://github.com/lyakaap/ISC21-Descriptor-Track-1st) and [paper](https://arxiv.org/abs/2112.04323) to learn more about contrastive learning and image instance retrieval.